In [19]:
from dateutil.parser import parse
import csv
from datetime import datetime

In [20]:
P = 0.2

In [21]:
class Citizen:
    def __init__(self, name, surname, birth_date):
        self.name = name
        self.surname = surname
        self.birth_date = birth_date

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, value):
        self._name = value

    @property
    def surname(self):
        return self._surname

    @surname.setter
    def surname(self, value):
        self._surname = value

    @property
    def birth_date(self):
        return self._birth_date

    @birth_date.setter
    def birth_date(self, value):
        self._birth_date = parse(value)


In [22]:
class BankAccount:
    def __init__(self, id, balance):
        self.id = id
        self.balance = balance

    def print_balance(self):
        print(f"Balance: {self.balance} UAH")

    def top_up(self, amount):
        self.balance += amount

    def withdraw(self, amount):
        if self.balance < amount:
            raise ValueError("Insufficient balance")

        self.balance -= amount

    @property
    def id(self):
        return self._id

    @id.setter
    def id(self, value):
        self._id = value

    @property
    def balance(self):
        return self._balance

    @balance.setter
    def balance(self, value):
        self._balance = value


In [23]:
class CreditBankAccount(BankAccount):
    def __init__(self, id, balance, credit_amount):
        super().__init__(id, balance)
        self.credit_amount = credit_amount

    @property
    def credit_amount(self):
        return self._credit_amount

    @credit_amount.setter
    def credit_amount(self, value):
        self._credit_amount = value

In [24]:
class Client(Citizen):
    def __init__(self, name, surname, birth_date, id, balance):
        super().__init__(name, surname, birth_date)
        self.bank_account = BankAccount(id, balance)

    def __str__(self):
        birth_date_string = self.birth_date.strftime('%d/%m/%Y')
        return (
                f"{self.name} {self.surname} ({birth_date_string})\n" +
                f"  Bank Account (id: {self.bank_account.id}) - {self.bank_account.balance} UAH"
        )

    def top_up(self, amount):
        self.bank_account.top_up(amount)

    def withdraw(self, amount):
        self.bank_account.withdraw(amount)

    @property
    def bank_account(self):
        return self._bank_account

    @bank_account.setter
    def bank_account(self, value):
        self._bank_account = value

In [25]:
class VipClient(Client):
    def __init__(self, name, surname, birth_date, id, balance, credit_amount, credit_account_balance):
        super().__init__(name, surname, birth_date, id, balance)
        self.credit_bank_account = CreditBankAccount(id, credit_account_balance, credit_amount)

    def __str__(self):
        birth_date_string = super().birth_date.strftime('%d/%m/%Y')
        return (
                f"{super().name} {super().surname} ({birth_date_string})\n" +
                f"  Bank Account (id: {super().bank_account.id}) - {super().bank_account.balance} UAH\n" +
                f"  Credit Bank Account (id: {self.credit_bank_account.id}) - {self.credit_bank_account.balance} UAH (credit amount: {self.credit_bank_account.credit_amount})\n" +
                f"  Total balance: {super().bank_account.balance + self.credit_bank_account.balance} UAH"
        )

    def issue_credit(self, amount):
        age_days = (datetime.datetime.now() - self.birth_date).days

        if 365 * 30 < age_days < 365 * 50:
            if amount > super().bank_account.balance * P:
                raise ValueError("Cannot issue credit of this amount")
            else:
                self.credit_bank_account.top_up(amount)
        else:
            if amount > super().bank_account.balance * P / 2:
                raise ValueError("Cannot issue credit of this amount")
            else:
                self.credit_bank_account.top_up(amount)

    def return_credit(self):
        credit_amount = self.credit_bank_account.credit_amount
        available_funds = self.credit_bank_account.balance + super().bank_account.balance

        if credit_amount > available_funds:
            raise ValueError("Insufficient funds")

        if self.credit_bank_account.balance >= credit_amount:
            self.credit_bank_account.withdraw(credit_amount)
        else:
            credit_amount -= self.credit_bank_account.balance
            self.credit_bank_account.withdraw(self.credit_bank_account.balance)
            super().bank_account.balance.withdraw(credit_amount)
            self.credit_bank_account.credit_amount = 0

In [26]:
# Read clients
clients = []
with open("clients.csv", "r") as file:
    reader = csv.DictReader(file)
    for row in reader:
        #name, surname, birth_date, id, balance
        clients.append(Client(row["name"], row["surname"], row["birth_date"], int(row["id"]), int(row["balance"])))

with open("vip_clients.csv", "r") as file:
    reader = csv.DictReader(file)
    for row in reader:
        #name, surname, birth_date, id, balance,  credit_amount, credit_account_balance
        clients.append(VipClient(row["name"], row["surname"], row["birth_date"], int(row["id"]), int(row["balance"]),
                                 int(row["credit_amount"]), int(row["credit_account_balance"])))

for client in clients:
    print(client)
    client.withdraw(50)

print()
for client in clients:
    print(client)

Sasha Dudynets (17/12/2004)
  Bank Account (id: 1) - 100 UAH
Yehor Furman (27/06/1994)
  Bank Account (id: 2) - 500 UAH
Rostyk Urdeichuk (17/12/2004)
  Bank Account (id: 3) - 100 UAH
  Credit Bank Account (id: 3) - 0 UAH (credit amount: 0)
  Total balance: 100 UAH
Danylo Yakumets (27/06/1994)
  Bank Account (id: 4) - 50000 UAH
  Credit Bank Account (id: 4) - 0 UAH (credit amount: 0)
  Total balance: 50000 UAH

Sasha Dudynets (17/12/2004)
  Bank Account (id: 1) - 50 UAH
Yehor Furman (27/06/1994)
  Bank Account (id: 2) - 350 UAH
Rostyk Urdeichuk (17/12/2004)
  Bank Account (id: 3) - 50 UAH
  Credit Bank Account (id: 3) - 0 UAH (credit amount: 0)
  Total balance: 50 UAH
Danylo Yakumets (27/06/1994)
  Bank Account (id: 4) - 49950 UAH
  Credit Bank Account (id: 4) - 0 UAH (credit amount: 0)
  Total balance: 49950 UAH


In [27]:
# Sorted clients
clients = sorted(clients, key=lambda client: client.surname)
for client in clients:
    print(client)

Sasha Dudynets (17/12/2004)
  Bank Account (id: 1) - 50 UAH
Yehor Furman (27/06/1994)
  Bank Account (id: 2) - 350 UAH
Rostyk Urdeichuk (17/12/2004)
  Bank Account (id: 3) - 50 UAH
  Credit Bank Account (id: 3) - 0 UAH (credit amount: 0)
  Total balance: 50 UAH
Danylo Yakumets (27/06/1994)
  Bank Account (id: 4) - 49950 UAH
  Credit Bank Account (id: 4) - 0 UAH (credit amount: 0)
  Total balance: 49950 UAH


In [28]:
# Clients with credits sorted by amount
clients_with_credits = filter(lambda client: isinstance(client, VipClient), clients)
clients_with_credits = sorted(clients_with_credits, key=lambda client: client.credit_bank_account.credit_amount)
for client in clients_with_credits:
    print(client)

Rostyk Urdeichuk (17/12/2004)
  Bank Account (id: 3) - 50 UAH
  Credit Bank Account (id: 3) - 0 UAH (credit amount: 0)
  Total balance: 50 UAH
Danylo Yakumets (27/06/1994)
  Bank Account (id: 4) - 49950 UAH
  Credit Bank Account (id: 4) - 0 UAH (credit amount: 0)
  Total balance: 49950 UAH
